# Interactive Visualization Lab

Complete the following set of exercises to solidify your knowledge of interactive visualization using Plotly, Cufflinks, and IPyWidgets.

In [1]:
import pandas as pd
import plotly.plotly as py
import cufflinks as cf
from ipywidgets import interact

cf.go_offline()

ModuleNotFoundError: No module named 'plotly'

In [ ]:
pip install plotly


  Obtaining dependency information for plotly from https://files.pythonhosted.org/packages/a8/07/72953cf70e3bd3a24cbc3e743e6f8539abe6e3e6d83c3c0c83426eaffd39/plotly-5.18.0-py3-none-any.whl.metadata
  Obtaining dependency information for tenacity>=6.2.0 from https://files.pythonhosted.org/packages/f4/f1/990741d5bb2487d529d20a433210ffa136a367751e454214013b441c4575/tenacity-8.2.3-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/15.6 MB ? eta -:--:--
   ---------------------------------------- 0.1/15.6 MB 1.7 MB/s eta 0:00:10
    --------------------------------------- 0.2/15.6 MB 2.5 MB/s eta 0:00:07
    --------------------------------------- 0.4/15.6 MB 2.5 MB/s eta 0:00:07
   - -------------------------------------- 0.5/15.6 MB 2.4 MB/s eta 0:00:07
   - -------------------------------------- 0.6/15.6 MB 2.7 MB/s eta 0:00:06
   - -------------------------------------- 0.7/15.6 MB 2.7 MB/s eta 0:00:06
   -- ------------------------------------- 0.9/15.6 MB 2.8 MB

## 1. Create an interactive bar chart showing total quantity and revenue by country (excluding United Kingdom) for the month of April 2011.

## 2. Create an interactive line chart showing quantity and revenue sold to France between January 1st and May 31st 2011.

## 3. Create an interactive scatter plot showing the relationship between average quantity (x-axis) and average unit price (y-axis) for the product PARTY BUNTING with the plot points color-coded by country (categories).

## 4. Create a set of interactive histograms showing the distributions of quantity per invoice for the following countries: EIRE, Germany, France, and Netherlands.

## 5. Create an interactive side-by-side bar chart showing the revenue by country listed below (bars) for each of the products listed below.

## 6. Create an interactive line chart showing quantity sold by day for the United Kingdom. Add drop-down boxes for Year and Month that allow you to filter the date range that appears in the chart.

## 7. Create an interactive scatter plot that plots number of invoices (x-axis) vs. number of customers (y-axis) and the plot points represent individual products. Add two sliders that control the x and y axis ranges.

## 8. Creat an interactive bar chart that shows revenue by product description. Add a text field widget that filters the results to show the product that contain the text entered in their description.